In [ ]:
from __future__ import unicode_literals
import youtube_dl
ydl_opts = {}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/watch?v=2JLiGSUohEo'])

In [1]:
from __future__ import unicode_literals
import ffmpeg
import youtube_dl
from pydub import AudioSegment

output_file_path="./audio_file/audio_sample"

ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl':  output_file_path + '.%(ext)s',   # 出力先パス
    'postprocessors': [
        {'key': 'FFmpegExtractAudio',
         'preferredcodec': 'mp3',                # 出力ファイル形式
         'preferredquality': '192'},             # 出力ファイルの品質
        {'key': 'FFmpegMetadata'},
    ],
}
# url = "https://www.youtube.com/watch?v=2JLiGSUohEo"
url = "https://www.youtube.com/watch?v=hR3tAX3pfZE"

ydl = youtube_dl.YoutubeDL(ydl_opts)

# 指定したパスに音声ファイルが格納される
_ = ydl.extract_info(url, download=True)

# 格納された音声ファイルをpydubで読み込む
audio = AudioSegment.from_mp3(output_file_path + '.mp3')

[youtube] hR3tAX3pfZE: Downloading webpage
[youtube] hR3tAX3pfZE: Downloading MPD manifest
[download] Destination: ./audio_file/audio_sample.m4a
[download] 100% of 1.23MiB in 00:2269KiB/s ETA 00:00
[ffmpeg] Correcting container in "./audio_file/audio_sample.m4a"
[ffmpeg] Destination: ./audio_file/audio_sample.mp3
Deleting original file ./audio_file/audio_sample.m4a (pass -k to keep)
[ffmpeg] Adding metadata to './audio_file/audio_sample.mp3'


In [2]:
from pydub import AudioSegment

# audio_file = 'YOUR_SONG.mp3'
# output_path = 'YOUR_SONG.wav'

audio_file = './audio_file/audio_sample.mp3'
output_path = 'audio_sample_w.wav'

audio = AudioSegment.from_mp3(audio_file)
audio.export(output_path, format='wav')

<_io.BufferedRandom name='audio_sample_w.wav'>

In [ ]:
from pydub import AudioSegment
import streamlit as st

byte_file = st.file_uploader('こちらからファイルを読み込み')
audio = AudioSegment.from_file(byte_file)
audio.export(output_path, format='wav')

In [6]:
import time
import azure.cognitiveservices.speech as speechsdk

def recognize_audio(output, speech_key, service_region, filename, recognize_time=100):
    """
    wav形式のデータから文字を起こす関数
    ---------------------------
    Parameters
    output: str
        音声から起こしたテキスト（再帰的に取得する）
    speech_key: str
        Azure Speech SDKのキー
    service_region: str
         Azure Speech SDKのリージョン名
    filename: str
        音声ファイルのパス
    recognize_time: int
        音声認識にかける時間（秒）
    """
    # Speech to Text 設定周り
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
    speech_config.speech_recognition_language="ja-JP"

    # 認識器の設定
    audio_input = speechsdk.AudioConfig(filename=filename)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_input)

    def recognized(evt):
        nonlocal output
        output += evt.result.text
        
    # 音声認識の実行
    # recognize_timeの時間、継続して文字起こしを行う
    speech_recognizer.recognized.connect(recognized)
    speech_recognizer.start_continuous_recognition()
    time.sleep(recognize_time)

    return output


filename = "audio_sample_w.wav"

# 音声ファイルの読み込み
sound = AudioSegment.from_file(filename, "wav")

# 情報の取得
# time = sound.duration_seconds # 再生時間(秒)
times = int(sound.duration_seconds)+10


speech_key = "f476c82f7e1242308bab15e791ea2600"
service_region = "japaneast"
output = ""

output = recognize_audio(output, speech_key, service_region, filename, recognize_time=times)

In [7]:
output

'いかがだったでしょうか？えっと、まあ、この講座でですね、ミニマムにやってまいりましたけれども、それでも立派なページが作れるようになったんじゃないでしょうかね。でえっと、まあ、今日今回覚えたことを使ってもらえば、ほぼまあどんなページでも作れるようになると。思っていいです。もちろんですね。まだまだ細かい設定はいろいろありますけれども、まあ一番ええ、第一の壁が一番高いと思うんですけど、そこはもう乗り越えましたので、あとはgoogleで調べてですね。なんとかなるんじゃないかなと思います。皆さんもですねええと、その他にもページを作っていただいて、いろいろと練習をしてみてください。でスキルハブの方ではえっと課題の方も出しますので、その課題もやってみてください。では以上になります。ありがとうございました。'

In [ ]:
with open("output.txt", "w", encoding="utf-8") as f:
    f.write(output)

In [ ]:
import azure.cognitiveservices.speech as speechsdk

speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)

# The language of the voice that speaks.
speech_config.speech_synthesis_voice_name='ja-JP-NanamiNeural'

speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)

# Get text from the console and synthesize to the default speaker.
#print("Enter some text that you want to speak >")
text = output

speech_synthesis_result = speech_synthesizer.speak_text_async(text).get()

if speech_synthesis_result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
    print("Speech synthesized for text [{}]".format(text))
elif speech_synthesis_result.reason == speechsdk.ResultReason.Canceled:
    cancellation_details = speech_synthesis_result.cancellation_details
    print("Speech synthesis canceled: {}".format(cancellation_details.reason))
    if cancellation_details.reason == speechsdk.CancellationReason.Error:
        if cancellation_details.error_details:
            print("Error details: {}".format(cancellation_details.error_details))
            print("Did you set the speech resource key and region values?")